
## 🌐 Scraping de Letras de Canciones Faltantes

El objetivo de este notebook es realizar scraping de letras de canciones faltantes para completar un dataset existente. Esto lo decidimos de último momento para evitar la pérdida de datos y asegurar que el dataset esté lo más completo posible dado el tiempo con el que contamos y que sería más complicado buscar datos nuevos para seguir alimentándolos. A continuación, se detallan los pasos y el código utilizado para lograr este objetivo.
  
Se intentaron varias formas, finalmente se optó por usar TunneIBear que modificase la dirección IP. Después del código final habrá ejemplos de códigos en fallido que se usaron. 

### Descripción del Código


1. **📂 Carga de Datasets**
    Se cargan los datasets necesarios para el proceso de scraping y actualización de letras de canciones.

2. **🌍 Detección de Idioma**
    Se configura la detección de idioma utilizando la librería `langdetect` para asegurar que las letras obtenidas estén en el idioma correcto y no tener que buscarlo después de nuevo.

3. **🔄 Funciones de Scraping**
    Se definen varias funciones para obtener letras de diferentes fuentes como LyricsFreak, Lyrics.com, AZLyrics, y Genius. Esto asegura que si una fuente falla, se pueda intentar con otra.

4. **🔁 Procesamiento de Filas**
    Se itera sobre cada fila del dataset para buscar y actualizar las letras faltantes. Si se encuentran letras, se detecta el idioma y se actualiza el dataset.

5. **💾 Guardado de Progreso**
    El progreso se guarda periódicamente para evitar la pérdida de datos en caso de interrupciones. Además, se registran los errores en un archivo separado.

6. **✅ Finalización del Proceso**
    Al finalizar, se guarda el dataset actualizado y se eliminan archivos temporales.
  
7. **🧹 Limpieza del texto para que las letras se guarden en el formato adecuado**
    Al finalizar, se guarda el dataset actualizado y se eliminan archivos temporales.

Este enfoque asegura que el dataset esté lo más completo y preciso posible, utilizando múltiples fuentes y técnicas de scraping para obtener las letras 
faltantes.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import random       # Para simular retrasos aleatorios
import re        # Para limpiar texto antes de detectar idioma
from langdetect import detect, DetectorFactory
from urllib.parse import quote

# 🔹 Configuración de detección de idioma
DetectorFactory.seed = 0

# 📌 Rutas de los archivos
original_file_path = os.path.join('..', 'src', 'df_no_lyrics.csv')
temp_file_path = os.path.join('..', 'data', '2901df_lyrics_temp.csv')
errors_file_path = os.path.join('..', 'data', 'not_found_lyrics.csv')
final_path = os.path.join('..', 'data', 'df_lyrics_final.csv')

# 🔹 Función para limpiar texto antes de detectar idioma
def clean_text_for_language_detection(text):
    """Limpia el texto eliminando solo símbolos extraños, pero manteniendo caracteres en español."""
    text = text.lower()
    text = re.sub(r"[^\w\sáéíóúüñ]", "", text)  # Mantiene caracteres en español
    return text.strip()

# 🔹 Función para detectar idioma con `langdetect`
def detect_language(text):
    """Detecta el idioma de la letra y muestra mensajes de depuración."""
    try:
        text = clean_text_for_language_detection(text)

        if len(text) < 20:
            print("⚠️ Texto demasiado corto para detectar idioma.")
            return "❓ (Texto muy corto)"

        detected_lang = detect(text)
        print(f"🌍 Idioma detectado: {detected_lang}")
        return detected_lang

    except Exception as e:
        print(f"⚠️ Error detectando idioma: {e}")
        return "❓"


# 🔹 Función para obtener letras de LyricsFreak
def get_lyrics_from_lyricsfreak(artist, song):
    """Busca letras en LyricsFreak."""
    base_url = "https://www.lyricsfreak.com"
    search_url = f"{base_url}/search.php?q={quote(artist + ' ' + song)}"

    try:
        response = requests.get(search_url, timeout=5)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            result_links = soup.find_all("a", class_="song")
            if result_links:
                lyrics_url = base_url + result_links[0]["href"]
                lyrics_response = requests.get(lyrics_url, timeout=5)
                lyrics_soup = BeautifulSoup(lyrics_response.text, "html.parser")
                lyrics_div = lyrics_soup.find("div", class_="lyrictxt")
                if lyrics_div:
                    return lyrics_div.get_text(separator="\n").strip()
    except Exception as e:
        print(f"⚠️ Error en LyricsFreak para {artist} - {song}: {e}")
        return None

    return None

# 🔹 Función para obtener letras de Lyrics.com
def get_lyrics_from_lyricsdotcom(artist, song):
    """Busca letras en Lyrics.com si LyricsFreak falla."""
    base_url = "https://www.lyrics.com"
    search_url = f"{base_url}/serp.php?st={quote(artist + ' ' + song)}&qtype=2"

    try:
        response = requests.get(search_url, timeout=5)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            result_links = soup.find_all("a", href=True)

            # Filtrar enlaces que apunten a letras de canciones
            for link in result_links:
                if "/lyric/" in link["href"]:
                    lyrics_url = base_url + link["href"]
                    lyrics_response = requests.get(lyrics_url, timeout=5)
                    lyrics_soup = BeautifulSoup(lyrics_response.text, "html.parser")
                    lyrics_div = lyrics_soup.find("pre", id="lyric-body-text")
                    if lyrics_div:
                        return lyrics_div.get_text(separator="\n").strip()

    except Exception as e:
        print(f"⚠️ Error en Lyrics.com para {artist} - {song}: {e}")
        return None

    return None

# 📌 Cargar dataset
if os.path.exists(temp_file_path):
    df = pd.read_csv(temp_file_path)
    print("✅ Archivo temporal cargado.")
else:
    df = pd.read_csv(original_file_path)
    print("📌 Cargando archivo original.")

errors = []
start_time = time.time()

# 🔁 Procesar cada fila
for index, row in df.iterrows():
    if pd.notnull(row.get('lyrics')) and pd.notnull(row.get('language')) and row['language'] != "❓":
        continue  

    artist, song = row['artist_name'], row['song_name']

    # 🎵 Intentar primero en LyricsFreak
    print(f"🔍 Buscando en LyricsFreak para {artist} - {song}...")
    lyrics = get_lyrics_from_lyricsfreak(artist, song)

    # 🔄 Si falla, intentar en Lyrics.com
    if not lyrics:
        print(f"🔄 Buscando en Lyrics.com para {artist} - {song}...")
        lyrics = get_lyrics_from_lyricsdotcom(artist, song)

    # 📌 Guardar letras si se encontraron
    if lyrics:
        lyrics = " ".join(lyrics.split())  # Limpiar texto
        df.at[index, 'lyrics'] = lyrics
        print(f"✅ Letras obtenidas: {artist} - {song}")
    else:
        print(f"❌ No se encontraron letras: {artist} - {song}")
        errors.append({'artist': artist, 'song': song})

    # 🌍 Detectar idioma
    df.at[index, 'language'] = detect_language(df.at[index, 'lyrics'])

    # 📂 Guardar progreso cada 10 iteraciones
    if index % 10 == 0:
        df.to_csv(temp_file_path, index=False, lineterminator='\n')

df.to_csv(final_path, index=False, lineterminator='\n')
print(f"✅ Proceso finalizado. Guardado en {final_path}.")

if errors:
    pd.DataFrame(errors).to_csv(errors_file_path, index=False)
    print(f"❌ {len(errors)} canciones sin letras. Guardado en {errors_file_path}.")


📌 Cargando archivo original.
🔍 Buscando en LyricsFreak para pupkulies & rebecca - la vie est belle...
🔄 Buscando en Lyrics.com para pupkulies & rebecca - la vie est belle...
❌ No se encontraron letras: pupkulies & rebecca - la vie est belle
⚠️ Error detectando idioma: 'numpy.float64' object has no attribute 'lower'
🔍 Buscando en LyricsFreak para lata mangeshkar - pyar kiya to darna kya...
🔄 Buscando en Lyrics.com para lata mangeshkar - pyar kiya to darna kya...
❌ No se encontraron letras: lata mangeshkar - pyar kiya to darna kya
⚠️ Error detectando idioma: 'numpy.float64' object has no attribute 'lower'
🔍 Buscando en LyricsFreak para the hives - civilizations dying...
🔄 Buscando en Lyrics.com para the hives - civilizations dying...
❌ No se encontraron letras: the hives - civilizations dying
⚠️ Error detectando idioma: 'numpy.float64' object has no attribute 'lower'
🔍 Buscando en LyricsFreak para the saturdays - dont let me dance alone...
🔄 Buscando en Lyrics.com para the saturdays - do

C:\Users\solan\AppData\Local\Temp\ipykernel_20732\529996292.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Oh, Cecilia, close your eyes and I will be with ya I'm in the song that's rising over ya, close your eyes and drift away Just the two of us broke up all the rules they made for us Play a simple game of genius if you live to fight another day Dream of all of the love you made It's there in your kaleidoscope eyes All my life and all that I've ever had, lies waiting here If you don't make up your mind, then how can I drift away If you don't let me be your guide then how can I drift away (drift away) Times are dangerous, think the future's got it in for us Play a simple game of genius if you live to fight another day Dream of all of the love you made It's there in your kaleidoscope eyes All my life and all that I've ever had, lies waiting here for you If you don't make up your mind, then how can 

✅ Letras obtenidas: noel gallagher - a simple game of genius
🌍 Idioma detectado: en
🔍 Buscando en LyricsFreak para richard thompson - wheely down...
🔄 Buscando en Lyrics.com para richard thompson - wheely down...
❌ No se encontraron letras: richard thompson - wheely down
⚠️ Error detectando idioma: 'float' object has no attribute 'lower'
🔍 Buscando en LyricsFreak para booker t jones - broken heart...
🔄 Buscando en Lyrics.com para booker t jones - broken heart...
❌ No se encontraron letras: booker t jones - broken heart
⚠️ Error detectando idioma: 'float' object has no attribute 'lower'
🔍 Buscando en LyricsFreak para joe diffie - show me a woman...
✅ Letras obtenidas: joe diffie - show me a woman
🌍 Idioma detectado: en
🔍 Buscando en LyricsFreak para pete murray - king tide...
🔄 Buscando en Lyrics.com para pete murray - king tide...
❌ No se encontraron letras: pete murray - king tide
⚠️ Error detectando idioma: 'float' object has no attribute 'lower'
🔍 Buscando en LyricsFreak para eric b

In [ ]:
import pandas as pd

# 📂 Cargar los datasets
df_large = pd.read_csv("src/functions b/df_fusion_scr.csv", low_memory=False)
df_modified = pd.read_csv("data/df_lyrics_final.csv", low_memory=False)

# 🔹 Definir clave de búsqueda y columnas a actualizar
key_column = 'recording_id'
columns_to_update = ['language', 'lyrics']

# 🔹 Fusionar los datasets asegurando que se actualicen correctamente
df_updated = df_large.merge(df_modified[[key_column] + columns_to_update], on=key_column, how="left", suffixes=("", "_new"))

# 🔹 Reemplazar valores **solo si están nulos (`NaN`) en `df_large`**
for col in columns_to_update:
    df_updated[col] = df_updated[col].where(df_updated[col].notna(), df_updated[col + "_new"])
    df_updated.drop(columns=[col + "_new"], inplace=True)  # Eliminar columnas auxiliares

# 📌 Guardar el dataset actualizado
df_updated.to_csv("df_lyrics_faltan_traduc.csv", index=False, encoding="utf-8")

print("✅ Actualización completada. Solo se llenaron valores nulos. Archivo guardado como 'df_fusion_scr.csv'.")


✅ Actualización completada. Solo se llenaron valores nulos. Archivo guardado como 'df_fusion_scr.csv'.


In [ ]:
import pandas as pd

# 📌 Cargar el dataset con las letras ya obtenidas
file_path = 'data/df_lyrics_final.csv'
df = pd.read_csv(file_path)

# 🔹 Función para limpiar el formato de las letras
def clean_lyrics(lyrics):
    if pd.isnull(lyrics):
        return lyrics  # No modificar si está vacío
    
    lyrics = lyrics.replace("\n", " ")  # Reemplazar saltos de línea por espacio
    lyrics = " ".join(lyrics.split())   # Eliminar espacios dobles
    return lyrics.strip()  # Eliminar espacios al inicio y final

# 🔄 Aplicar la limpieza a todas las letras
df['lyrics'] = df['lyrics'].apply(clean_lyrics)

# 📌 Guardar el dataset limpio
cleaned_file_path = 'data/final_scrapeado.csv'
df.to_csv(cleaned_file_path, index=False)

print(f"✅ Letras formateadas correctamente y guardadas en {cleaned_file_path}.")


#### Otras pruebas varias que se hicieron

In [2]:
def scrape_azlyrics(url, headers):
    """Hace scraping en AZLyrics usando session para evitar bloqueos."""
    try:
        session = requests.Session()
        session.headers.update(headers)  # Usar headers realistas
        response = session.get(url, timeout=10)

        if response.status_code == 404:
            print(f"⚠️ Página no encontrada en AZLyrics: {url}")
            return None
        elif response.status_code == 403:
            print(f"🚫 Acceso prohibido en AZLyrics (403): {url}")
            return None  # No intentar más si está bloqueado

        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        lyrics_div = soup.find_all("div", class_=False)  # AZLyrics usa div sin clase
        if lyrics_div:
            return lyrics_div[0].get_text(separator="\n").strip()

    except requests.exceptions.RequestException as e:
        print(f"⚠️ Error en AZLyrics: {e}")
        return None


In [ ]:
import requests
import zipfile
import os

def download_chromedriver(destination_folder):
    # Endpoint para obtener la última versión estable de ChromeDriver
    endpoint = "https://googlechromelabs.github.io/chrome-for-testing/last-known-good-versions.json"
    response = requests.get(endpoint)
    data = response.json()

    # Extraer la última versión estable para Win64
    stable_version = data["channels"]["Stable"]["version"]
    download_url = f"https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/{stable_version}/win64/chromedriver-win64.zip"

    # Descargar el archivo ZIP
    response = requests.get(download_url)
    zip_path = os.path.join(destination_folder, "chromedriver.zip")
    with open(zip_path, "wb") as file:
        file.write(response.content)

    # Extraer el archivo ZIP
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(destination_folder)

    # Eliminar el archivo ZIP después de la extracción
    os.remove(zip_path)

    print(f"ChromeDriver {stable_version} descargado y extraído en: {destination_folder}")

# Carpeta donde guardar ChromeDriver
destination = "C:\\webdrivers\\chrome"
os.makedirs(destination, exist_ok=True)
download_chromedriver(destination)


ChromeDriver 132.0.6834.110 descargado y extraído en: C:\webdrivers\chrome


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# Ruta completa al archivo chromedriver.exe
driver_service = Service(r'C:\\webdrivers\\chrome\\chromedriver.exe')

# Configurar el navegador Chrome
driver = webdriver.Chrome(service=driver_service)

# Abre una página web para probar
driver.get("https://www.google.com")

print("¡Navegador abierto correctamente!")
driver.quit()


¡Navegador abierto correctamente!


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from langdetect import detect

# Configurar Selenium con ChromeDriver
def setup_driver():
    options = Options()
    options.add_argument("--headless")  # Ejecutar en modo headless
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")  # User-Agent personalizado
    driver_service = Service('C:\\webdrivers\\chrome\\chromedriver.exe')  # Cambiar ruta al chromedriver
    driver = webdriver.Chrome(service=driver_service, options=options)
    return driver

# Función para obtener letras y traducciones de Musixmatch
def get_lyrics_and_translation_musixmatch(driver, artist, song):
    url = f"https://www.musixmatch.com/lyrics/{artist.replace(' ', '-')}/{song.replace(' ', '-')}"
    driver.get(url)
    try:
        # Esperar hasta que la letra esté visible
        lyrics_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'mxm-lyrics__content'))
        )
        original_lyrics = lyrics_element.text.strip() if lyrics_element else None

        # Intentar obtener traducción (si está disponible)
        try:
            translation_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="mxm-lyrics__content" and @data-translation]'))
            )
            translated_lyrics = translation_element.text.strip() if translation_element else None
        except:
            translated_lyrics = None  # Si no hay traducción disponible

        return original_lyrics, translated_lyrics

    except Exception as e:
        print(f"Error al obtener datos para {artist} - {song}: {e}")
        return None, None

# Función para detectar el idioma
def detect_language(text):
    try:
        return detect(text)  # Detecta el idioma (por ejemplo, 'en', 'es', 'fr')
    except Exception as e:
        print(f"No se pudo detectar el idioma: {e}")
        return None

# Cargar el dataset
df = pd.read_csv(r'C:\Users\solan\Downloads\get_data_from_songs\data\final_idiomas_traducido.csv')

# Filtrar filas con valores nulos en 'lyrics' o 'translated_lyrics'
null_lyrics = df[(df['lyrics'].isnull()) | ((df['language'].isnull()) & df['translated_lyrics'].isnull())]

# Configurar Selenium
driver = setup_driver()

# Iterar sobre las filas y actualizar
for index, row in null_lyrics.iterrows():
    artist = row['artist_name']
    song = row['song_name']
    
    # Obtener letras y traducciones
    original_lyrics, translated_lyrics = get_lyrics_and_translation_musixmatch(driver, artist, song)
    
    # Actualizar 'lyrics' si se encontró la letra
    if original_lyrics and pd.isnull(row['lyrics']):
        df.at[index, 'lyrics'] = original_lyrics
        
        # Detectar el idioma si 'language' está vacío
        if pd.isnull(row['language']):
            detected_language = detect_language(original_lyrics)
            df.at[index, 'language'] = detected_language
    
    # Actualizar 'translated_lyrics' si cumple condiciones
    if translated_lyrics and pd.isnull(row['translated_lyrics']) and row['language'] != 'en':
        df.at[index, 'translated_lyrics'] = translated_lyrics

# Cerrar Selenium
driver.quit()

# Guardar el dataset actualizado
df.to_csv(r'C:\Users\solan\Downloads\get_data_from_songs\data\final_idiomas_traducido_updated.csv', index=False)

print("Proceso completado. Dataset actualizado guardado.")


Error al obtener datos para the saturdays - dont let me dance alone: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7554FCC05+28741]
	(No symbol) [0x00007FF75545FED0]
	(No symbol) [0x00007FF7552F4FBA]
	(No symbol) [0x00007FF755349256]
	(No symbol) [0x00007FF75534948C]
	(No symbol) [0x00007FF755393267]
	(No symbol) [0x00007FF75536FE6F]
	(No symbol) [0x00007FF75538FFE1]
	(No symbol) [0x00007FF75536FBD3]
	(No symbol) [0x00007FF755339F43]
	(No symbol) [0x00007FF75533B2D1]
	GetHandleVerifier [0x00007FF75582DE3D+3375741]
	GetHandleVerifier [0x00007FF75584096A+3452330]
	GetHandleVerifier [0x00007FF75583671D+3410781]
	GetHandleVerifier [0x00007FF7555C653B+854395]
	(No symbol) [0x00007FF75546BE0F]
	(No symbol) [0x00007FF755467714]
	(No symbol) [0x00007FF7554678AD]
	(No symbol) [0x00007FF755456189]
	BaseThreadInitThunk [0x00007FFD5D00E8D7+23]
	RtlUserThreadStart [0x00007FFD5EC1FBCC+44]

Error al obtener datos para the hives - square one here i come: Message: 
Stacktrace:
	GetHandleVerifier [

KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_lyrics_from_genius(artist, song):
    # Crear URL de Genius
    base_url = "https://genius.com"
    search_url = f"{base_url}/{artist.replace(' ', '-')}-{song.replace(' ', '-')}-lyrics"
    
    response = requests.get(search_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        # Buscar el contenedor de letras
        lyrics_div = soup.find("div", class_="Lyrics__Container")
        if lyrics_div:
            lyrics = lyrics_div.get_text(separator="\n").strip()
            return lyrics
    return None

# Ejemplo de uso
lyrics = get_lyrics_from_genius("Taylor Swift", "Lover")
print(lyrics)


None


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from langdetect import detect
import os
import time

# Función para extraer letras de AZLyrics
def get_lyrics_from_azlyrics(artist, song):
    base_url = "https://www.azlyrics.com/lyrics"
    url = f"{base_url}/{artist.replace(' ', '').lower()}/{song.replace(' ', '').lower()}.html"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            # Buscar letras en el contenedor principal
            lyrics_div = soup.find("div", class_=False, id=False)
            if lyrics_div:
                return lyrics_div.get_text(separator="\n").strip()
        return None
    except Exception as e:
        print(f"Error al obtener letras para {artist} - {song}: {e}")
        return None

# Función para detectar el idioma de las letras
def detect_language(text):
    try:
        return detect(text)  # Devuelve el código de idioma, por ejemplo: 'en', 'es', 'fr'
    except Exception as e:
        print(f"No se pudo detectar el idioma: {e}")
        return None

# Archivos
original_file_path = 'data/duplicados_lyrics.csv'
temp_file_path = 'data/duplicados_temp.csv'
errors_file_path = 'data/not_found_songs_duplicados.csv'

# Cargar el dataset
if os.path.exists(temp_file_path):
    df = pd.read_csv(temp_file_path)
    print("Archivo temporal cargado. Continuando desde el progreso guardado.")
else:
    df = pd.read_csv(original_file_path)
    print("No se encontró archivo temporal. Iniciando desde el archivo original.")

# Lista para registrar errores
errors = []

# Total de filas a procesar
total_rows = df[(df['lyrics'].isnull()) | (df['language'].isnull())].shape[0]
processed_rows = 0
save_interval = 2000  # Guardar cada 2000 filas

# Iterar sobre las filas del dataset
for index, row in df.iterrows():
    # Verificar si ya se procesó (si `lyrics` y `language` están llenos, continuar)
    if pd.notnull(row['lyrics']) and pd.notnull(row['language']):
        continue

    # Obtener información de la fila
    artist = row['artist_name']
    song = row['song_name']

    # Obtener letras
    if pd.isnull(row['lyrics']):
        lyrics = get_lyrics_from_azlyrics(artist, song)
        if lyrics:
            df.at[index, 'lyrics'] = lyrics  # Rellenar letras
            print(f"Letras encontradas para {artist} - {song}.")
        else:
            print(f"No se encontraron letras para {artist} - {song}.")
            errors.append({'artist': artist, 'song': song})  # Registrar en errores
    
    # Detectar idioma
    if pd.notnull(df.at[index, 'lyrics']) and pd.isnull(row['language']):
        detected_language = detect_language(df.at[index, 'lyrics'])
        if detected_language:
            df.at[index, 'language'] = detected_language
            print(f"Idioma detectado para {artist} - {song}: {detected_language}.")
    
    # Incrementar contador de filas procesadas
    processed_rows += 1

    # Guardar el progreso cada 2000 filas
    if processed_rows % save_interval == 0:
        df.to_csv(temp_file_path, index=False)
        print(f"Progreso guardado: {processed_rows}/{total_rows} filas procesadas ({(processed_rows / total_rows) * 100:.2f}%).")
        # Guardar errores en un archivo CSV
        pd.DataFrame(errors).to_csv(errors_file_path, index=False)

    # Esperar para evitar bloqueos
    time.sleep(1)

# Guardar el dataset final
df.to_csv(original_file_path, index=False)
print(f"Proceso completado. Dataset actualizado guardado en {original_file_path}.")

# Guardar errores
if errors:
    pd.DataFrame(errors).to_csv(errors_file_path, index=False)
    print(f"Se registraron {len(errors)} canciones sin letras. Guardado en {errors_file_path}.")

# Eliminar el archivo temporal
if os.path.exists(temp_file_path):
    os.remove(temp_file_path)
    print("Archivo temporal eliminado.")
